# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbbf43c2370>
├── 'a' --> tensor([[ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210]])
└── 'x' --> <FastTreeValue 0x7fbbf43c24c0>
    └── 'c' --> tensor([[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                        [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                        [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]])

In [4]:
t.a

tensor([[ 1.6972,  0.9094,  0.3213],
        [-0.2175, -0.4795,  0.3210]])

In [5]:
%timeit t.a

77.3 ns ± 0.918 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbbf43c2370>
├── 'a' --> tensor([[-1.4968,  0.0285,  0.2799],
│                   [-1.4032,  0.6922,  0.6504]])
└── 'x' --> <FastTreeValue 0x7fbbf43c24c0>
    └── 'c' --> tensor([[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                        [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                        [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]])

In [7]:
%timeit t.a = new_value

92.7 ns ± 1.94 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6972,  0.9094,  0.3213],
               [-0.2175, -0.4795,  0.3210]]),
    x: Batch(
           c: tensor([[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                      [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                      [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6972,  0.9094,  0.3213],
        [-0.2175, -0.4795,  0.3210]])

In [11]:
%timeit b.a

73 ns ± 1.85 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1079,  0.2556, -1.3852],
               [ 1.9806, -0.2393,  0.5683]]),
    x: Batch(
           c: tensor([[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                      [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                      [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]]),
       ),
)

In [13]:
%timeit b.a = new_value

669 ns ± 16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.09 µs ± 38.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.4 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

276 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

261 µs ± 9.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbb31761730>
├── 'a' --> tensor([[[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]],
│           
│                   [[ 1.6972,  0.9094,  0.3213],
│                    [-0.2175, -0.4795,  0.3210]]])
└── 'x' --> <FastTreeValue 0x7fbbf93d8610>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.3 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbb31761cd0>
├── 'a' --> tensor([[ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210],
│                   [ 1.6972,  0.9094,  0.3213],
│                   [-0.2175, -0.4795,  0.3210]])
└── 'x' --> <FastTreeValue 0x7fbbf43c7fd0>
    └── 'c' --> tensor([[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                        [ 1.3147e-01,  4.1653e-01,  5.0101e-01,

In [23]:
%timeit t_cat(trees)

43.7 µs ± 508 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

93.3 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                       [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                       [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]],
              
                      [[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                       [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                       [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]],
              
                      [[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                       [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                       [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]],
              
                      [[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                       [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                       [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01]],
      

In [26]:
%timeit Batch.stack(batches)

108 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                      [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                      [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01],
                      [ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                      [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                      [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01],
                      [ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                      [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                      [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01],
                      [ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.3839e+00],
                      [ 1.3147e-01,  4.1653e-01,  5.0101e-01,  1.4170e+00],
                      [-1.2813e+00,  2.3267e+00, -1.2615e-03, -5.0658e-01],
                      [ 2.0242e-02, -3.2378e-01, -1.3066e+00,  2.38

In [28]:
%timeit Batch.cat(batches)

202 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

612 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
